In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# -------------------------------------------------------------------
# FUNCTION TO SCRAPE TECH SECTOR TICKERS FROM STOCKANALYSIS.COM
# -------------------------------------------------------------------
def get_tickers():
    url = "https://stockanalysis.com/stocks/sector/technology/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find('table')
        if table is None:
            raise ValueError("Could not find table on webpage")

        tickers = []
        rows = table.find_all('tr')[1:]

        for row in rows:
            cols = row.find_all('td')
            if cols:
                ticker = cols[1].text.strip()
                tickers.append(ticker)

        return tickers

    except Exception as e:
        print(f"Error scraping tickers: {str(e)}")
        return []

# -------------------------------------------------------------------
# EXECUTE SCRAPING FUNCTION AND SAVE OUTPUT
# -------------------------------------------------------------------
tickers = get_tickers()
df = pd.DataFrame({'Ticker': tickers})
df.to_csv('tickers.csv', index=False)

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm

# -------------------------------------------------------------------
# CLASS FOR FETCHING CASH FLOW DATA FROM SEC EDGAR API
# -------------------------------------------------------------------
class SECDataFetcher:
    def __init__(self, email):
        self.headers = {
            'User-Agent': f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) {email}'
        }
        self.base_url = "https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json"
        self.company_tickers_url = "https://www.sec.gov/files/company_tickers.json"

    def get_cik_mapping(self):
        response = requests.get(self.company_tickers_url, headers=self.headers)
        data = response.json()
        cik_map = {entry['ticker']: str(entry['cik_str']).zfill(10) for entry in data.values()}
        return cik_map

    def get_cash_flows(self, cik):
        try:
            url = self.base_url.format(cik)
            response = requests.get(url, headers=self.headers)

            if response.status_code != 200:
                return None

            data = response.json()
            facts = data['facts']['us-gaap']

            years = range(datetime.now().year - 5, datetime.now().year)
            cash_flows = {year: {'Operating_CF': None, 'Investing_CF': None, 'Financing_CF': None} for year in years}

            for tag, key in [
                ('NetCashProvidedByUsedInOperatingActivities', 'Operating_CF'),
                ('NetCashProvidedByUsedInInvestingActivities', 'Investing_CF'),
                ('NetCashProvidedByUsedInFinancingActivities', 'Financing_CF')
            ]:
                if tag in facts:
                    for entry in facts[tag].get('units', {}).get('USD', []):
                        if 'form' in entry and entry['form'] == '10-K':
                            year = int(entry['end'][:4])
                            if year in cash_flows:
                                cash_flows[year][key] = entry['val']

            return cash_flows

        except Exception as e:
            print(f"Error fetching data for CIK {cik}: {str(e)}")
            return None

# -------------------------------------------------------------------
# FUNCTION TO CLASSIFY LIFE CYCLE STAGES BASED ON CASH FLOW SIGNS
# -------------------------------------------------------------------
def classify_lifecycle(cash_flows):
    classifications = {}

    for year, flows in cash_flows.items():
        if None in flows.values():
            classifications[year] = "Data Missing"
            continue

        o, i, f = flows['Operating_CF'], flows['Investing_CF'], flows['Financing_CF']
        o_sign = '+' if o > 0 else '-'
        i_sign = '+' if i > 0 else '-'
        f_sign = '+' if f > 0 else '-'

        patterns = {
            ('-', '-', '+'): "Introduction",
            ('+', '-', '+'): "Growth",
            ('+', '-', '-'): "Maturity",
            ('+', '+', '-'): "Shake-out",
            ('+', '+', '+'): "Shake-out",
            ('-', '-', '-'): "Shake-out",
            ('-', '+', '-'): "Decline",
            ('-', '+', '+'): "Decline",
        }

        classifications[year] = patterns[(o_sign, i_sign, f_sign)]

    return classifications

# -------------------------------------------------------------------
# MAIN EXECUTION FUNCTION
# -------------------------------------------------------------------
def main():
    email = "sofyasavina3@gmail.com"
    fetcher = SECDataFetcher(email)

    tickers_df = pd.read_csv("tickers.csv")
    tickers = tickers_df['Ticker'].str.upper().unique().tolist()

    print("Getting CIK mapping...")
    cik_map = fetcher.get_cik_mapping()

    results = []
    print("Fetching cash flow data for last 5 years...")

    for ticker in tqdm(tickers):
        if ticker in cik_map:
            cik = cik_map[ticker]
            cash_flows = fetcher.get_cash_flows(cik)

            if cash_flows:
                classifications = classify_lifecycle(cash_flows)
                for year in cash_flows.keys():
                    results.append({
                        'Ticker': ticker,
                        'Year': year,
                        'Life_Cycle_Stage': classifications[year],
                        **cash_flows[year]
                    })

            time.sleep(0.1)  # Respect SEC rate limits

    results_df = pd.DataFrame(results)
    results_df.to_csv('company_lifecycles.csv', index=False)

    print("\nLife Cycle Stage Distribution:")
    print(results_df.groupby(['Year', 'Life_Cycle_Stage']).size())

    return results_df

# -------------------------------------------------------------------
# RUN MAIN FUNCTION
# -------------------------------------------------------------------
if __name__ == "__main__":
    results_df = main()

Getting CIK mapping...
Fetching cash flow data for last 5 years...


  1%|          | 5/775 [00:02<06:30,  1.97it/s]

Error fetching data for CIK 0001046179: 'us-gaap'


  1%|          | 7/775 [00:03<06:07,  2.09it/s]

Error fetching data for CIK 0001000184: 'us-gaap'


...

Error fetching data for CIK 0001964630: 'us-gaap'


 93%|█████████▎| 723/775 [05:49<00:19,  2.65it/s]

Error fetching data for CIK 0001976443: 'us-gaap'


100%|██████████| 775/775 [06:11<00:00,  2.09it/s]

Error fetching data for CIK 0001888151: 'us-gaap'

Life Cycle Stage Distribution:
Year  Life_Cycle_Stage
2020  Data Missing        200
      Decline              25
      Growth              125
      Introduction        140
      Maturity            173
      Shake-out            49
2021  Data Missing        154
      Decline              25
      Growth              148
      Introduction        163
      Maturity            183
      Shake-out            39
2022  Data Missing        146
      Decline              47
      Growth               98
      Introduction        133
      Maturity            196
      Shake-out            92
2023  Data Missing        141
      Decline              58
      Growth               95
      Introduction         91
      Maturity            247
      Shake-out            80
2024  Data Missing        180
      Decline              52
      Growth               88
      Introduction         79
      Maturity            239
      Shake-out          

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm

# -------------------------------------------------------------------
# CLASS FOR FETCHING MULTIPLE FINANCIAL TAGS FROM SEC EDGAR API
# -------------------------------------------------------------------
class SECDataFetcher:
    def __init__(self, email):
        self.headers = {
            'User-Agent': f'Mozilla/5.0 (compatible; ThesisResearchBot/1.0; {email})'
        }
        self.base_url = "https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json"
        self.tickers_url = "https://www.sec.gov/files/company_tickers.json"

    def get_cik_mapping(self):
        response = requests.get(self.tickers_url, headers=self.headers)
        data = response.json()
        return {entry['ticker'].upper(): str(entry['cik_str']).zfill(10) for entry in data.values()}

    def get_multi_tag_data(self, cik, tag_list):
        url = self.base_url.format(cik)
        response = requests.get(url, headers=self.headers)
        if response.status_code != 200:
            return {}

        data = response.json()
        facts = data.get('facts', {}).get('us-gaap', {})
        results = {}

        for tag in tag_list:
            entries = facts.get(tag, {}).get('units', {}).get('USD', [])
            for entry in entries:
                if entry.get('form') == '10-K' and entry.get('fp') == 'FY':
                    year = entry.get('fy')
                    val = entry.get('val')
                    end = entry.get('end')
                    if year and 2019 <= year <= 2024 and val:
                        end_dt = datetime.strptime(end, "%Y-%m-%d")
                        if year not in results or end_dt > results[year]['end']:
                            results[year] = {'val': val, 'end': end_dt}

        return {year: item['val'] for year, item in results.items()}

# -------------------------------------------------------------------
# LOAD TICKERS AND INITIALIZE FETCHER
# -------------------------------------------------------------------
tickers_df = pd.read_csv("tickers.csv")
tickers = tickers_df['Ticker'].str.upper().unique().tolist()

email = "sofyasavina3@gmail.com"
fetcher = SECDataFetcher(email)
cik_map = fetcher.get_cik_mapping()

# -------------------------------------------------------------------
# DEFINE TAGS FOR FINANCIAL VARIABLES
# -------------------------------------------------------------------
tags = {
    'revenue': [
        'RevenueFromContractWithCustomerExcludingAssessedTax',
        'Revenues',
        'SalesRevenueNet'
    ],
    'net_income': ['NetIncomeLoss'],
    'total_assets': ['Assets'],
    'current_assets': ['AssetsCurrent'],
    'current_liabilities': ['LiabilitiesCurrent'],
    'short_term_debt': [
        'DebtCurrent',
        'ShortTermBorrowings',
        'CurrentPortionOfLongTermDebtAndCapitalLeaseObligations'
    ],
    'long_term_debt': [
        'LongTermDebtNoncurrent',
        'LongTermBorrowings',
        'LongTermDebt'
    ],
    'other_liabilities': [
        'OtherLiabilities',
        'OperatingLeaseLiabilityNoncurrent',
        'DeferredRevenueNoncurrent',
        'DeferredTaxLiabilitiesNoncurrent'
    ],
    'cash': ['CashAndCashEquivalentsAtCarryingValue'],
    'r_and_d': ['ResearchAndDevelopmentExpense'],
    'ocf': ['NetCashProvidedByUsedInOperatingActivities'],
    'capex': [
        'CapitalExpenditures',
        'PaymentsToAcquirePropertyPlantAndEquipment',
        'PaymentsForCapitalExpenditures',
        'CapitalExpendituresIncurredButNotYetPaid'
    ]
}

# -------------------------------------------------------------------
# FETCH AND CALCULATE FINANCIAL RATIOS
# -------------------------------------------------------------------
results = []

for ticker in tqdm(tickers):
    cik = cik_map.get(ticker)
    if not cik:
        continue

    data = {key: fetcher.get_multi_tag_data(cik, tag) for key, tag in tags.items()}

    for year in range(2020, 2025):
        try:
            rev = data['revenue'].get(year)
            prev_rev = data['revenue'].get(year - 1)
            ni = data['net_income'].get(year)
            ta = data['total_assets'].get(year)
            ta_prev = data['total_assets'].get(year - 1)
            avg_assets = (ta + ta_prev) / 2 if ta and ta_prev else None

            ca = data['current_assets'].get(year)
            cl = data['current_liabilities'].get(year)
            std = data['short_term_debt'].get(year, 0)
            ltd = data['long_term_debt'].get(year, 0)
            other_liab = data['other_liabilities'].get(year, 0)
            cash = data['cash'].get(year)
            rnd = data['r_and_d'].get(year)
            ocf = data['ocf'].get(year)
            capex = data['capex'].get(year)

            total_debt = sum(x for x in [std, ltd, other_liab] if x is not None)

            row = {
                'ticker': ticker,
                'year': year,
                'revenue': rev,
                'net_income': ni,
                'total_assets': ta,
                'current_ratio': ca / cl if ca and cl else None,
                'debt_assets': total_debt / ta if total_debt and ta else None,
                'asset_turnover': rev / avg_assets if rev and avg_assets else None,
                'roa': ni / avg_assets if ni and avg_assets else None,
                'revenue_growth': (rev - prev_rev) / prev_rev if rev and prev_rev else None,
                'r_and_d_ratio': rnd / rev if rnd and rev else None,
                'capex_ratio': capex / ta if capex and ta else None,
                'net_income_margin': ni / rev if ni and rev else None,
                'ocf_margin': ocf / rev if ocf and rev else None
            }
            results.append(row)
        except Exception as e:
            print(f"Error processing {ticker} {year}: {e}")

    time.sleep(0.1)

# -------------------------------------------------------------------
# SAVE OUTPUT TO CSV
# -------------------------------------------------------------------
df = pd.DataFrame(results)
df.to_csv("raw_financial

100%|██████████| 775/775 [39:52<00:00,  3.09s/it]

✅ Financial ratio dataset saved to 'raw_financial_ratios.csv'


In [ ]:
import requests
import pandas as pd
import time
import yfinance as yf
from tqdm import tqdm
from datetime import datetime

# -------------------------------------------------------------------
# CLASS FOR FETCHING SHARES OUTSTANDING FROM SEC EDGAR API
# -------------------------------------------------------------------
class SECDataFetcher:
    def __init__(self, email):
        self.headers = {
            'User-Agent': f'Mozilla/5.0 (compatible; {email})'
        }
        self.base_url = "https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json"
        self.company_tickers_url = "https://www.sec.gov/files/company_tickers.json"

    def get_cik_mapping(self):
        response = requests.get(self.company_tickers_url, headers=self.headers)
        data = response.json()
        return {entry['ticker']: str(entry['cik_str']).zfill(10) for entry in data.values()}

    def get_shares_outstanding(self, cik):
        try:
            url = self.base_url.format(cik)
            response = requests.get(url, headers=self.headers)
            if response.status_code != 200:
                return None

            data = response.json()
            facts = data.get('facts', {}).get('us-gaap', {})
            shares_data = {}

            for tag in ['CommonStockSharesOutstanding', 'EntityCommonStockSharesOutstanding']:
                if tag in facts:
                    units = facts[tag].get('units', {})
                    for unit_type in units:
                        if 'share' in unit_type.lower():
                            for entry in units[unit_type]:
                                if entry.get('form') == '10-K':
                                    fy = int(entry.get('fy', 0))
                                    if 2020 <= fy <= 2024:
                                        end_date = entry['end']
                                        shares = entry['val']
                                        if fy not in shares_data:
                                            shares_data[fy] = []
                                        shares_data[fy].append(shares)

            return {fy: sum(vals)/len(vals) for fy, vals in shares_data.items()} if shares_data else None

        except Exception as e:
            print(f"Error fetching shares for CIK {cik}: {e}")
            return None

# -------------------------------------------------------------------
# FUNCTION TO GET AVERAGE STOCK PRICE USING YFINANCE
# -------------------------------------------------------------------
def get_average_price(ticker, fiscal_year):
    try:
        fiscal_year = int(fiscal_year)
        end_date = datetime(fiscal_year, 12, 31)
        start_date = end_date - pd.DateOffset(days=30)

        history = yf.Ticker(ticker).history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

        if history.empty:
            return None

        return history['Close'].mean()

    except Exception as e:
        print(f"Error fetching average price for {ticker} in {fiscal_year}: {e}")
        return None

# -------------------------------------------------------------------
# FUNCTION TO CALCULATE MARKET CAP
# -------------------------------------------------------------------
def main():
    email = "sofyasavina3@gmail.com"
    fetcher = SECDataFetcher(email)
    cik_map = fetcher.get_cik_mapping()

    tickers_df = pd.read_csv("tickers.csv")
    tickers = tickers_df['Ticker'].str.upper().unique().tolist()

    results = []
    for ticker in tqdm(tickers):
        if ticker not in cik_map:
            continue

        cik = cik_map[ticker]
        shares_dict = fetcher.get_shares_outstanding(cik)

        if shares_dict:
            for fy, avg_shares in shares_dict.items():
                avg_price = get_average_price(ticker, fy)
                market_cap = avg_shares * avg_price if avg_price is not None else None
                results.append({
                    'Ticker': ticker,
                    'Fiscal Year': fy,
                    'Avg_Shares_Outstanding': avg_shares,
                    'Avg_Stock_Price': avg_price,
                    'Market_Capitalization': market_cap,
                })

        time.sleep(0.1)

    df = pd.DataFrame(results)
    df.sort_values(by=['Ticker', 'Fiscal Year'], inplace=True)
    df.to_csv("market_cap.csv", index=False)
    print("✅ Saved to 'market_cap.csv'")
    return df

# -------------------------------------------------------------------
# RUN MAIN FUNCTION
# -------------------------------------------------------------------
if __name__ == "__main__":
    main()

 33%|███▎      | 253/775 [03:07<03:54,  2.23it/s]ERROR:yfinance:$LIF: possibly delisted; no price data found  (1d 2022-12-01 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1669870800, endDate = 1672462800")
ERROR:yfinance:$LIF: possibly delisted; no price data found  (1d 2023-12-01 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1701406800, endDate = 1703998800")
 37%|███▋      | 284/775 [03:31<05:12,  1.57it/s]ERROR:yfinance:$CORZ: possibly delisted; no price data found  (1d 2022-12-01 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1669870800, endDate = 1672462800")
ERROR:yfinance:$CORZ: possibly delisted; no price data found  (1d 2023-12-01 -> 2023-12-31) (Yahoo error = "Data doesn't exist for startDate = 1701406800, endDate = 1703998800")
 39%|███▉      | 304/775 [03:47<06:15,  1.25it/s]ERROR:yfinance:$DBD: possibly delisted; no price data found  (1d 2020-12-01 -> 2020-12-31) (Yahoo error = "Data doesn't exist for startDate = 1

✅ Saved to 'market_cap.csv'


In [ ]:
import requests
import pandas as pd
import time
import yfinance as yf
from datetime import datetime
from tqdm import tqdm

# -------------------------------------------------------------------
# CLASS FOR FETCHING FINANCIAL DATA FROM SEC EDGAR API
# -------------------------------------------------------------------
class SECDataFetcher:
    def __init__(self, email):
        self.headers = {
            'User-Agent': f'Mozilla/5.0 (compatible; {email})'
        }
        self.base_url = "https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json"
        self.company_tickers_url = "https://www.sec.gov/files/company_tickers.json"

    def get_cik_mapping(self):
        response = requests.get(self.company_tickers_url, headers=self.headers)
        data = response.json()
        return {entry['ticker']: str(entry['cik_str']).zfill(10) for entry in data.values()}

    def get_financials(self, cik):
        try:
            url = self.base_url.format(cik)
            response = requests.get(url, headers=self.headers)
            if response.status_code != 200:
                return None

            data = response.json()
            facts = data['facts']['us-gaap']

            financials = {year: {'TotalDebt': None, 'CashEquivalents': None} for year in range(2020, 2025)}

            debt_tags = {
                'LongTermDebtNoncurrent': 'long_term',
                'DebtCurrent': 'short_term'
            }

            for tag, label in debt_tags.items():
                if tag in facts:
                    for entry in facts[tag]['units'].get('USD', []):
                        if entry.get('form') == '10-K' and 'fy' in entry:
                            fy = int(entry['fy'])
                            if fy in financials:
                                if financials[fy]['TotalDebt'] is None:
                                    financials[fy]['TotalDebt'] = 0
                                financials[fy]['TotalDebt'] += entry['val']

            if 'CashAndCashEquivalentsAtCarryingValue' in facts:
                for entry in facts['CashAndCashEquivalentsAtCarryingValue']['units'].get('USD', []):
                    if entry.get('form') == '10-K' and 'fy' in entry:
                        fy = int(entry['fy'])
                        if fy in financials:
                            financials[fy]['CashEquivalents'] = entry['val']

            return financials

        except Exception as e:
            print(f"Error fetching financials for CIK {cik}: {e}")
            return None

# -------------------------------------------------------------------
# FUNCTION TO CALCULATE ENTERPRISE VALUE
# -------------------------------------------------------------------
def main():
    email = "sofyasavina3@gmail.com"
    fetcher = SECDataFetcher(email)

    tickers_df = pd.read_csv("tickers.csv")
    tickers = tickers_df['Ticker'].str.upper().unique().tolist()

    print("Getting CIK mapping...")
    cik_map = fetcher.get_cik_mapping()

    print("Loading Market Cap data...")
    market_cap_df = pd.read_csv('market_cap.csv')
    market_cap_dict = {
        (row['Ticker'], row['Fiscal Year']): row['Market_Capitalization']
        for _, row in market_cap_df.iterrows()
    }

    results = []
    print("Fetching Enterprise Value data for 2020–2024...")
    for ticker in tqdm(tickers):
        if ticker not in cik_map:
            continue

        cik = cik_map[ticker]
        financial_data = fetcher.get_financials(cik)

        if financial_data:
            for year in range(2020, 2025):
                market_cap = market_cap_dict.get((ticker, year))
                total_debt = financial_data[year]['TotalDebt']
                cash_eq = financial_data[year]['CashEquivalents']

                if market_cap is not None and total_debt is not None and cash_eq is not None:
                    enterprise_value = market_cap + total_debt - cash_eq
                else:
                    enterprise_value = None

                results.append({
                    'Ticker': ticker,
                    'Year': year,
                    'Market_Capitalization': market_cap,
                    'Total_Debt': total_debt,
                    'Cash_Equivalents': cash_eq,
                    'Enterprise_Value': enterprise_value
                })

        time.sleep(0.1)

    df = pd.DataFrame(results)
    df.to_csv('enterprise_value.csv', index=False)
    print("✅ Data saved to 'enterprise_value.csv'")
    return df

# -------------------------------------------------------------------
# RUN MAIN FUNCTION
# -------------------------------------------------------------------
if __name__ == "__main__":
    main()

Getting CIK mapping...
Loading Market Cap data...
Fetching Enterprise Value data for 2020–2024...


  1%|          | 5/775 [00:02<06:02,  2.13it/s]

Error fetching financials for CIK 0001046179: 'us-gaap'


  1%|          | 7/775 [00:03<05:10,  2.47it/s]

Error fetching financials for CIK 0001000184: 'us-gaap'


...

Error fetching financials for CIK 0001964630: 'us-gaap'


 93%|█████████▎| 723/775 [05:27<00:19,  2.70it/s]

Error fetching financials for CIK 0001976443: 'us-gaap'


100%|██████████| 775/775 [05:47<00:00,  2.23it/s]

Error fetching financials for CIK 0001888151: 'us-gaap'
✅ Data saved to 'enterprise_value.csv'


In [ ]:
import pandas as pd

# -------------------------------------------------------------------
# LOAD RAW FINANCIALS AND ENTERPRISE VALUE DATASETS
# -------------------------------------------------------------------
raw_df = pd.read_csv("raw_financials.csv")
ev_df = pd.read_csv("enterprise_value.csv")

print("Raw Financials Columns:", raw_df.columns.tolist())
print("Enterprise Value Columns:", ev_df.columns.tolist())

# -------------------------------------------------------------------
# STANDARDIZE COLUMN NAMES FOR MERGING
# -------------------------------------------------------------------
raw_df.rename(columns=lambda x: x.strip().capitalize(), inplace=True)
ev_df.rename(columns=lambda x: x.strip().capitalize(), inplace=True)

# -------------------------------------------------------------------
# MERGE AND COMPUTE EV/SALES
# -------------------------------------------------------------------
if 'Ticker' in raw_df.columns and 'Year' in raw_df.columns and 'Enterprise_value' in ev_df.columns:
    merged = pd.merge(raw_df, ev_df[['Ticker', 'Year', 'Enterprise_value']], on=['Ticker', 'Year'], how='left')
    merged['Ev_sales'] = merged['Enterprise_value'] / merged['Revenue']
    merged.drop(columns='Enterprise_value', inplace=True)

    merged.to_csv("financials_with_ev_sales.csv", index=False)
    print("✅ Merged dataset with EV/Sales saved as 'financials_with_ev_sales.csv'")
else:
    print("❌ Ticker or Year or Enterprise_value column not found in one of the datasets.")

Raw Financials Columns: ['ticker', 'year', 'revenue', 'net_income', 'total_assets', 'current_ratio', 'debt_assets', 'asset_turnover', 'roa', 'revenue_growth', 'r_and_d_ratio', 'capex_ratio', 'net_income_margin', 'ocf_margin']
Enterprise Value Columns: ['Ticker', 'Year', 'Market_Capitalization', 'Total_Debt', 'Cash_Equivalents', 'Enterprise_Value']
✅ Merged dataset with EV/Sales saved as 'financials_with_ev_sales.csv'


In [ ]:
import pandas as pd

# -------------------------------------------------------------------
# LOAD LIFECYCLE AND FINANCIAL DATASETS
# -------------------------------------------------------------------
lifecycles_df = pd.read_csv("company_lifecycles.csv")
financials_df = pd.read_csv("financials_with_ev_sales.csv")

# -------------------------------------------------------------------
# DROP UNNECESSARY COLUMNS
# -------------------------------------------------------------------
columns_to_drop = ['Operating_CF', 'Investing_CF', 'Financing_CF']
lifecycles_df = lifecycles_df.drop(columns=[col for col in columns_to_drop if col in lifecycles_df.columns])
financials_df = financials_df.drop(columns='Net_income')

# -------------------------------------------------------------------
# MERGE DATASETS ON TICKER AND YEAR
# -------------------------------------------------------------------
final_df = pd.merge(financials_df, lifecycles_df, on=['Ticker', 'Year'], how='left')

# -------------------------------------------------------------------
# SAVE FINAL MERGED DATASET
# -------------------------------------------------------------------
final_df.to_csv("raw_dataset.csv", index=False)
print("✅ Final dataset saved as 'raw_dataset.csv'")

✅ Final dataset saved as 'raw_dataset.csv'
